In [93]:
import torch
print(torch.cuda.is_available())
import numpy as np

True


In [94]:

model = "x35"
rcs_file = "../output/" + model + '_rcs.csv'
x35_data = np.genfromtxt (rcs_file, delimiter=",")
x35_data = x35_data[:,0:4]
x35_data[:,0]=0
print(x35_data.shape)
print(x35_data[0:10,:])

model = "hawker_900"
rcs_file = "../output/" + model + '_rcs.csv'
hawker_data = np.genfromtxt (rcs_file, delimiter=",")
hawker_data = hawker_data[:,0:4]
hawker_data[:,0]=1
print(hawker_data.shape)
print(hawker_data[0:10,:])


model = "a380_v1"
rcs_file = "../output/" + model + '_rcs.csv'
a380_data = np.genfromtxt (rcs_file, delimiter=",")
a380_data = a380_data[:,0:4]
a380_data[:,0]=2
print(a380_data.shape)
print(a380_data[0:10,:])


result = np.vstack((x35_data, hawker_data, a380_data))
print(result.shape)

result = result[~np.isnan(result[:,3])]

np.savetxt("source_data.csv", result, delimiter=",")



(65341, 4)
[[ 0.       0.       0.           nan]
 [ 0.       0.       1.       7.41526]
 [ 0.       0.       2.       8.21722]
 [ 0.       0.       3.       9.97604]
 [ 0.       0.       4.      10.2831 ]
 [ 0.       0.       5.      11.996  ]
 [ 0.       0.       6.      11.3603 ]
 [ 0.       0.       7.       9.88235]
 [ 0.       0.       8.      10.3555 ]
 [ 0.       0.       9.      10.2486 ]]
(65341, 4)
[[ 1.      0.      0.     32.1608]
 [ 1.      0.      1.     34.5449]
 [ 1.      0.      2.     30.6187]
 [ 1.      0.      3.     32.6597]
 [ 1.      0.      4.     22.0051]
 [ 1.      0.      5.     33.8001]
 [ 1.      0.      6.     33.3004]
 [ 1.      0.      7.     29.7234]
 [ 1.      0.      8.     34.3146]
 [ 1.      0.      9.     33.2395]]
(65341, 4)
[[ 2.      0.      0.         nan]
 [ 2.      0.      1.         nan]
 [ 2.      0.      2.     37.5076]
 [ 2.      0.      3.     39.8445]
 [ 2.      0.      4.     52.5334]
 [ 2.      0.      5.     32.743 ]
 [ 2.      0.  

In [95]:
from sklearn.model_selection import train_test_split

X = result[:, 1:]
y = result[:, 0].astype(int)

# Split the data into training and testing sets
# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# X_train and y_train contain the features and labels for the training set
# X_test and y_test contain the features and labels for the testing set


In [96]:
print(X_train[0:10])
print(X_test[0:10])

[[265.       62.       30.5036 ]
 [ 53.       30.       10.772  ]
 [ 10.      120.       14.0601 ]
 [285.      145.       29.9993 ]
 [281.      174.       32.8816 ]
 [339.       52.        9.92841]
 [249.      106.       12.0878 ]
 [201.      168.       23.97   ]
 [  4.      163.       29.5356 ]
 [270.       69.       18.5205 ]]
[[ 83.       79.       21.4693 ]
 [245.      120.       22.9201 ]
 [ 72.      112.       24.3053 ]
 [242.      142.       15.4415 ]
 [103.      144.       24.8959 ]
 [ 26.      139.       18.9498 ]
 [172.       16.       24.3271 ]
 [303.      177.       28.0846 ]
 [  0.       71.        3.76352]
 [342.       94.       14.7645 ]]


In [97]:
print(y_train[0:10])
print(y_test[0:10])

[2 1 2 2 1 1 0 1 2 1]
[0 2 2 1 2 2 2 1 0 2]


In [98]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)


In [99]:
import torch.nn as nn

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class AircraftClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AircraftClassifier, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

input_dim = 3  # phi, theta, RCS
hidden_dim = 128
num_classes = len(np.unique(y))  # Number of unique identifier numbers

model = AircraftClassifier(input_dim, hidden_dim, num_classes)
print(num_classes)

Using cuda device
3


In [104]:
import torch.optim as optim
import torch
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
scheduler = StepLR(optimizer, step_size=50, gamma=0.9)

# Training loop
epochs = 3000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/3000, Loss: 4.394686222076416
Epoch 2/3000, Loss: 1.7743974924087524
Epoch 3/3000, Loss: 1.1684634685516357
Epoch 4/3000, Loss: 1.2407500743865967
Epoch 5/3000, Loss: 1.0032165050506592
Epoch 6/3000, Loss: 0.9355776309967041
Epoch 7/3000, Loss: 0.9122474789619446
Epoch 8/3000, Loss: 0.8953492045402527
Epoch 9/3000, Loss: 0.8671440482139587
Epoch 10/3000, Loss: 0.8601195812225342
Epoch 11/3000, Loss: 0.8703271746635437
Epoch 12/3000, Loss: 0.8609082698822021
Epoch 13/3000, Loss: 0.8314235806465149
Epoch 14/3000, Loss: 0.808230996131897
Epoch 15/3000, Loss: 0.8076921701431274
Epoch 16/3000, Loss: 0.823618471622467
Epoch 17/3000, Loss: 0.8345422744750977
Epoch 18/3000, Loss: 0.8263078927993774
Epoch 19/3000, Loss: 0.8042262196540833
Epoch 20/3000, Loss: 0.7841448783874512
Epoch 21/3000, Loss: 0.776968240737915
Epoch 22/3000, Loss: 0.7814493179321289
Epoch 23/3000, Loss: 0.7889019846916199
Epoch 24/3000, Loss: 0.7914949655532837
Epoch 25/3000, Loss: 0.7870773077011108
Epoch 26/3000

In [106]:
# Convert the test data to PyTorch tensors
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Get the model's predictions
with torch.no_grad():
    predictions = model(X_test_tensor)
    _, predicted_labels = torch.max(predictions, 1)
    accuracy = (predicted_labels == y_test_tensor).float().mean()
    print(f'Test Accuracy: {accuracy.item()}')

Test Accuracy: 0.7617971897125244


In [107]:
torch.save(model, 'aircraft_classifier_model.pth')

In [ ]:
loaded_model = torch.load('aircraft_classifier_model.pth')